In [1]:
import os
import simulation
import pynbody
import pandas as pd
import matplotlib.pylab as plt
import matplotlib
import numpy as np
import ipywidgets
import tqdm
from simulation.sfh_in_box import sfh, plot_sfh, plot_binned_sfh, bin_sfh
from astropy import constants as c
from astropy.table import Table
from simulation.luminosity import surface_brightness, color_plot
from simulation.angmom import sideon
from simulation.derived import feh, mgfe, gas_metals

In [5]:
from simulation.parsers.parse_trace import parse_dens_trace

In [8]:
self = simulation.Simulation("/media/michele/My Book/Michele/MySimulations/MovingBox/np/mb.41002_p300_a800_r600_t9.56/out")

loading simulation: /media/michele/My Book/Michele/MySimulations/MovingBox/np/mb.41002_p300_a800_r600_t9.56/out
Found parameter file
Found compiler file
Found 445 snapshots
Loading cosmological parameters
Found trace file
Found dens_temp_trace file
dens_temp_trace.txt file is non-monotonic. Skipping for now


In [81]:
self.times.min(), self.times.max()

(SimArray(9.35767451, 'Gyr'), SimArray(13.68939532, 'Gyr'))

In [11]:
diff = self.trace.t.diff()
np.digitize(self.times_header, self.trace.t[diff > 0])

ValueError: bins must be monotonically increasing or decreasing

In [109]:
self.trace.t[(diff <0)]

241507     10.5317
423355     11.1021
605703     11.8780
791405     12.9376
953184     13.6765
1044241    13.8916
Name: t, dtype: float64

In [128]:
restart_points = self.trace.t[(diff <0)]
restart_points

241507     10.5317
423355     11.1021
605703     11.8780
791405     12.9376
953184     13.6765
1044241    13.8916
Name: t, dtype: float64

In [129]:
before_restart_points = self.trace.t.loc[restart_points.index-1]
before_restart_points

241506     10.5320
423354     11.1025
605702     11.8790
791404     12.9393
953183     13.6767
1044240    13.8919
Name: t, dtype: float64

In [134]:
before_restart_points

241506     10.5320
423354     11.1025
605702     11.8790
791404     12.9393
953183     13.6767
1044240    13.8919
Name: t, dtype: float64

In [130]:
restart_points.iloc[0]

10.5317

In [131]:
self.trace.loc[241507-5:241507+25]

,t,x,y,z,vx,vy,vz,ax,ay,az,r,v,a
241502,10.5320,-217.874,247.230,0.864436,-981.029,-416.428,-0.265303,-0.002102,0.013156,-0.002986,329.533762,1065.753371,0.013653
241503,10.5320,-217.879,247.229,0.864435,-981.024,-416.434,-0.265346,-0.002102,0.013156,-0.002986,329.536317,1065.751113,0.013654
241504,10.5320,-217.883,247.227,0.864433,-981.019,-416.440,-0.265388,-0.002098,0.013160,-0.002984,329.537461,1065.748855,0.013657
241505,10.5320,-217.887,247.225,0.864432,-981.014,-416.445,-0.265431,-0.002098,0.013161,-0.002984,329.538606,1065.746207,0.013657
241506,10.5320,-217.891,247.223,0.864431,-981.009,-416.451,-0.265473,-0.002098,0.013162,-0.002984,329.539750,1065.743949,0.013658
241507,10.5317,-217.555,247.366,0.864522,-981.414,-415.998,-0.262029,0.012400,0.004814,-0.002996,329.425050,1065.939888,0.013635
241508,10.5317,-217.559,247.364,0.864520,-981.409,-416.003,-0.262071,0.012397,0.004813,-0.002996,329.426190,1065.937235,0.013632
241509,10.5317,-217.563,247.362,0.864519,-981.403,-416.009,-0.262114,0.012398,0.004813,-0.002996,329.427329,1065.934053,0.013633
241510,10.5317,-217.567,247.361,0.864518,-981.398,-416.015,-0.262157,0.012398,0.004813,-0.002997,329.429220,1065.931791,0.013633
241511,10.5317,-217.571,247.359,0.864517,-981.393,-416.020,-0.262199,0.012403,0.004815,-0.003005,329.430360,1065.929139,0.013640


In [139]:
' or '.join(['index > {} and t > {}'.format(idx, t) for idx, t in zip(before_restart_points.index, before_restart_points.values)])

'index > 241506 and t > 10.532 or index > 423354 and t > 11.1025 or index > 605702 and t > 11.879000000000001 or index > 791404 and t > 12.9393 or index > 953183 and t > 13.6767 or index > 1044240 and t > 13.8919'

In [132]:
self.trace.query('index > {} and t > {}'.format(before_restart_points.index[0], before_restart_points.iloc[0])).index[0]

241596

In [140]:
end_idx = [self.trace.query('index > {} and t > {}'.format(idx, t)).index[0] for idx, t in zip(before_restart_points.index, before_restart_points.values)] 
end_idx

[241596, 423551, 605901, 791628, 953233, 1044330]

In [156]:
before_restart_points.index, end_idx

(Int64Index([241506, 423354, 605702, 791404, 953183, 1044240], dtype='int64'),
 [241596, 423551, 605901, 791628, 953233, 1044330])

In [166]:
na = self.trace.copy()
for a, b in zip(before_restart_points.index, end_idx):
    na = na.drop(self.trace.index[slice(a,b)])

In [167]:
na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1080273 entries, 0 to 1081122
Data columns (total 13 columns):
t     1080273 non-null float64
x     1080273 non-null float64
y     1080273 non-null float64
z     1080273 non-null float64
vx    1080273 non-null float64
vy    1080273 non-null float64
vz    1080273 non-null float64
ax    1080273 non-null float64
ay    1080273 non-null float64
az    1080273 non-null float64
r     1080273 non-null float64
v     1080273 non-null float64
a     1080273 non-null float64
dtypes: float64(13)
memory usage: 115.4 MB


In [165]:
self.trace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081123 entries, 0 to 1081122
Data columns (total 13 columns):
t     1081123 non-null float64
x     1081123 non-null float64
y     1081123 non-null float64
z     1081123 non-null float64
vx    1081123 non-null float64
vy    1081123 non-null float64
vz    1081123 non-null float64
ax    1081123 non-null float64
ay    1081123 non-null float64
az    1081123 non-null float64
r     1081123 non-null float64
v     1081123 non-null float64
a     1081123 non-null float64
dtypes: float64(13)
memory usage: 107.2 MB


In [168]:
np.digitize(self.times_header, na.t)

array([    990,    2391,    4186,    6218,    8076,   10435,   12780,
         15018,   16835,   18435,   19617,   20795,   21976,   23157,
         24339,   25520,   26852,   28376,   30604,   32742,   34637,
         36609,   37844,   39093,   41455,   43811,   45901,   47783,
         49782,   51923,   53252,   54433,   55615,   56796,   57977,
         59159,   60313,   61793,   63536,   64994,   66964,   68941,
         70637,   72365,   73619,   74801,   77024,   78603,   79793,
         82088,   84207,   86261,   87639,   89643,   92005,   93885,
         95474,   97153,   99514,  103086,  107588,  110287,  112643,
        115006,  117369,  121139,  124810,  129299,  131965,  136689,
        140823,  145376,  150077,  153407,  156399,  159219,  163305,
        167341,  172058,  176242,  179467,  184181,  188699,  191660,
        195125,  199412,  203970,  208236,  212957,  217420,  221429,
        225234,  228488,  232390,  234781,  238499,  241018,  243373,
        245736,  248

In [162]:
self.trace.drop(self.trace.index[slice(241506,241596)])

,t,x,y,z,vx,vy,vz,ax,ay,az,r,v,a
0,9.56206,599.864,0.004696,-7.152080e-11,-389.096,554.82,-0.000017,0.000990,-0.000668,-0.002273,599.864000,677.658417,0.002568
1,9.56207,599.861,0.009393,-2.862650e-10,-389.103,554.82,-0.000034,0.000989,-0.000671,-0.002279,599.861000,677.662436,0.002574
2,9.56208,599.858,0.014089,-6.444160e-10,-389.111,554.82,-0.000051,0.000989,-0.000671,-0.002279,599.858000,677.667029,0.002574
3,9.56209,599.855,0.018786,-1.145980e-09,-389.118,554.82,-0.000068,0.000989,-0.000671,-0.002279,599.855000,677.671049,0.002574
4,9.56210,599.851,0.023482,-1.790950e-09,-389.126,554.82,-0.000085,0.000989,-0.000671,-0.002279,599.851000,677.675642,0.002574
5,9.56211,599.848,0.028178,-2.579350e-09,-389.133,554.82,-0.000102,0.000990,-0.000671,-0.002279,599.848001,677.679662,0.002574
6,9.56211,599.845,0.032875,-3.511170e-09,-389.141,554.82,-0.000119,0.000990,-0.000671,-0.002279,599.845001,677.684256,0.002574
7,9.56212,599.841,0.037571,-4.586210e-09,-389.148,554.82,-0.000135,0.000990,-0.000669,-0.002273,599.841001,677.688275,0.002568
8,9.56213,599.838,0.042267,-5.804280e-09,-389.155,554.82,-0.000152,0.000990,-0.000669,-0.002273,599.838001,677.692295,0.002568
9,9.56214,599.835,0.046964,-7.165380e-09,-389.163,554.82,-0.000169,0.000990,-0.000669,-0.002273,599.835002,677.696889,0.002568


In [151]:
self.trace.t.is_monotonic_increasing

False

In [160]:
mono = self.trace.drop(self.trace.index[slice(a, b)] for a, b in zip(before_restart_points.index, end_idx))

TypeError: unhashable type: 'RangeIndex'

In [86]:
self.trace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081123 entries, 0 to 1081122
Data columns (total 13 columns):
t     1081123 non-null float64
x     1081123 non-null float64
y     1081123 non-null float64
z     1081123 non-null float64
vx    1081123 non-null float64
vy    1081123 non-null float64
vz    1081123 non-null float64
ax    1081123 non-null float64
ay    1081123 non-null float64
az    1081123 non-null float64
r     1081123 non-null float64
v     1081123 non-null float64
a     1081123 non-null float64
dtypes: float64(13)
memory usage: 107.2 MB


In [84]:
df_mono = self.trace.drop_duplicates(subset=['t', 'r'], inplace=False)

In [85]:
df_mono.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1080341 entries, 0 to 1081122
Data columns (total 13 columns):
t     1080341 non-null float64
x     1080341 non-null float64
y     1080341 non-null float64
z     1080341 non-null float64
vx    1080341 non-null float64
vy    1080341 non-null float64
vz    1080341 non-null float64
ax    1080341 non-null float64
ay    1080341 non-null float64
az    1080341 non-null float64
r     1080341 non-null float64
v     1080341 non-null float64
a     1080341 non-null float64
dtypes: float64(13)
memory usage: 115.4 MB


In [ ]:
self.trace.drop_duplicates

In [77]:
df_mono.t[~(df_mono.t.diff() >=0)]

0    9.56206
Name: t, dtype: float64

In [89]:
self.trace[~self.trace.duplicated(subset=['t', 'r'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1080341 entries, 0 to 1081122
Data columns (total 13 columns):
t     1080341 non-null float64
x     1080341 non-null float64
y     1080341 non-null float64
z     1080341 non-null float64
vx    1080341 non-null float64
vy    1080341 non-null float64
vz    1080341 non-null float64
ax    1080341 non-null float64
ay    1080341 non-null float64
az    1080341 non-null float64
r     1080341 non-null float64
v     1080341 non-null float64
a     1080341 non-null float64
dtypes: float64(13)
memory usage: 115.4 MB


In [ ]:
DataFrame.duplicated(subset=None

In [34]:
self.trace.query('t==11.8780')

,t,x,y,z,vx,vy,vz,ax,ay,az,r,v,a
605500,11.878,-596.491,-496.061,-1.99642,197.728,-392.760,-2.53161,-0.006244,-0.001084,0.002985,775.810553,439.730814,0.007005
605501,11.878,-596.490,-496.063,-1.99643,197.730,-392.758,-2.53160,-0.006244,-0.001084,0.002985,775.811063,439.729927,0.007006
605502,11.878,-596.489,-496.064,-1.99644,197.732,-392.757,-2.53158,-0.006245,-0.001084,0.002985,775.810934,439.729933,0.007006
605503,11.878,-596.488,-496.066,-1.99645,197.734,-392.755,-2.53157,-0.006245,-0.001084,0.002985,775.811444,439.729046,0.007006
605504,11.878,-596.488,-496.068,-1.99646,197.736,-392.753,-2.53155,-0.006245,-0.001084,0.002985,775.812723,439.728159,0.007006
605505,11.878,-596.487,-496.069,-1.99647,197.738,-392.752,-2.53154,-0.006256,-0.001075,0.002985,775.812593,439.728165,0.007015
605506,11.878,-596.486,-496.071,-1.99648,197.740,-392.750,-2.53152,-0.006256,-0.001075,0.002985,775.813103,439.727278,0.007015
605507,11.878,-596.485,-496.073,-1.99649,197.742,-392.748,-2.53151,-0.006242,-0.001067,0.002985,775.813613,439.726391,0.007001
605508,11.878,-596.484,-496.074,-1.99650,197.744,-392.746,-2.53149,-0.006242,-0.001067,0.002985,775.813484,439.725504,0.007001
605509,11.878,-596.483,-496.076,-1.99652,197.746,-392.745,-2.53148,-0.006242,-0.001067,0.002985,775.813994,439.725510,0.007001


In [19]:
def make_monotonic(series):
    pass

SyntaxError: invalid syntax (<ipython-input-19-7508397ce182>, line 1)